# Jane Street Market Prediction: Exploratory Data Analysis

The aim of this notebook is to provide an initial EDA for the competition's training data, also sharing some ideas on strategies to solve the challenge.

## Data loading

Load the training data.

In [ ]:
def is_kaggle_gpu_enabled():
    
    """
    Return whether GPU is enabled in the running Kaggle kernel
    """

    from tensorflow.python.client import device_lib

    # when only CPU is enabled the list shows two CPU entries, otherwise there are more, listing GPU as well
    return len(device_lib.list_local_devices()) > 2

In [ ]:
is_kaggle_gpu_enabled()

In [ ]:
import pandas as pd

data_path = "/kaggle/input/jane-street-market-prediction/train.csv"

if is_kaggle_gpu_enabled():
    
    import cudf

    cudf_data = cudf.read_csv(data_path, index_col='ts_id')
    train_data = cudf_data.to_pandas()
    del cudf_data
    
else: 
    train_data = pd.read_csv(data_path, index_col='ts_id')

## Data filtering

* Operations with non-positive weight are irrelevant for scoring, so ignored.

* For RAM saving, numeric (int or float) data is made to use 32 bits instead of 64. 

In [ ]:
import numpy as np

train_data = train_data[train_data["weight"] > 0]
train_data.reset_index(drop=True, inplace=True)

train_data = train_data.astype({col: np.float32 for col in train_data.select_dtypes('float64').columns})
train_data = train_data.astype({col: np.int32 for col in train_data.select_dtypes('int64').columns})

## Features and target definition

Set which are the predictors (x) and the variable to predict (y).

In [ ]:
# the target variable is the action (1 to make the trading operation and 0 to skip it);
# in training, the operation is considered positive if it has positive return (the future time horizons are not used in evaluation metric)
full_y_train = (train_data['resp'] >= 0).astype('int')

# the predictor variables are the feature columns
x_cols = ['feature_' + str(i) for i in range(0, 130)]
full_x_train = train_data[x_cols]

# series used later
weight_series = train_data['weight']
resp_series = train_data['resp']
date_series = train_data['date']

## Utility score function

The mathematical formulation comes from the competition's [evaluation page](https://www.kaggle.com/c/jane-street-market-prediction/overview/evaluation). The very efficient implementation posted by [Yirun Zhang](https://www.kaggle.com/gogo827jz) in [Jane Street: Super Fast Utility Score Function](https://www.kaggle.com/gogo827jz/jane-street-super-fast-utility-score-function) is used here.

In [ ]:
def utility_score(dates, weights, resps, actions):
    # adapted from from https://www.kaggle.com/gogo827jz/jane-street-super-fast-utility-score-function
    count_i = dates.max() + 1 - dates.min()
    p_i = np.bincount(dates, weights * resps * actions)
    t = np.sum(p_i) / np.sqrt(np.sum(p_i ** 2)) * np.sqrt(250 / count_i)
    u = np.clip(t, 0, 6) * np.sum(p_i)
    return u

In [ ]:
train_perfect_score = utility_score(date_series, weight_series, resp_series, full_y_train)
print("Perfect training score:", train_perfect_score)

## Weight inspection and strategy

The competition's [evaluation page](https://www.kaggle.com/c/jane-street-market-prediction/overview/evaluation) states that the outcome of a trade is the result of multiplying its weight by the response (or return), which is zeroed out if the action is 0, and kept otherwise (if the transaction is applied). Hence, it seems possible that the weight is the invested value and the response is a factor that gives the won/loss amount (if positive/negative respectively).

Trades have an average weight of roughly 1, with some trades being clearly less relevant (25% have less than 0.38), and others being higher (another 25% of the operations have a weight three times greater than the mean). 

Almost 9% of samples have a weight greater than 10. It is not a large proportion, but might be numerous enough to consider using techniques that aim at getting high-weight samples classified right, since they have higher impact on the score, either positively or negatively (depending on the response). It might be reckless to weight these samples as much more important when modelling (e.g. having higher impact in a loss function) and not taking proper care of the remaining ones (which are less valuable but much more numerous). Other alternatives might be more suitable, such as building two models, one for high-weight samples and one for standard/low-weight ones. However, it is not obvious if the amount of samples in the high-weight group will be enough to train a strong model, considering the low signal-to-noise ratio of the dataset.

In [ ]:
weight_series.describe()

In [ ]:
weight_over_10_count = weight_series[weight_series > 10].count()
print("Samples with weight over 10: {} ({}% of training data)".format(weight_over_10_count, round(weight_over_10_count / weight_series.shape[0] * 100, 3)))

In [ ]:
weight_over_100_count = weight_series[weight_series > 100].count()
print("Samples with weight over 100: {} ({}% of training data)".format(weight_over_100_count, round(weight_over_100_count / weight_series.shape[0] * 100, 3)))

In [ ]:
import plotly.express as px

px.histogram(weight_series, nbins=50, title='Weight distribution')

## Response inspection

The mean response is almost zero, so a trivial baseline approach of blindly applying all operations (always using action 1) would give close to no score.

No operation obtains a (proportionally) massive loss/win, since the response never reaches 0.6, or 60%; the products/instruments might be from a market that is not as volatile as cryptocurrences, for instance. Actually, in the great majority of cases (over 93%) the win or loss is within 5%, and in 37% of the samples within 0.05%. It seems that the market presents relatively low risks along with relatively low gain margins.

In [ ]:
resp_series.describe()

In [ ]:
abs(resp_series).describe()

In [ ]:
resp_abs_0_005_count = resp_series[abs(resp_series) <= 0.005].count()
print("Samples with response in [-0.005, 0.005]: {} ({}% of training data)".format(resp_abs_0_005_count, round(resp_abs_0_005_count / weight_series.shape[0] * 100, 3)))

In [ ]:
resp_abs_0_05_count = resp_series[abs(resp_series) <= 0.05].count()
print("Samples with response in [-0.05, 0.05]: {} ({}% of training data)".format(resp_abs_0_05_count, round(resp_abs_0_05_count / weight_series.shape[0] * 100, 3)))

In [ ]:
resp_out_abs_1_count = resp_series[abs(resp_series) > 0.1].count()
print("Samples with response over 0.1 or below -0.1: {} ({}% of training data)".format(resp_out_abs_1_count, round(resp_out_abs_1_count / weight_series.shape[0] * 100, 3)))

In [ ]:
px.histogram(resp_series, nbins=40, title='Response distribution')

## Class distribution

The two classes have almost the same number of samples in the training data, so the data distribution is balanced (assuming that this is still true in the unseen test data). This should allow to avoid using techniques such as data augmentation or class weighting when preparing classification models. It also implies that any baseline obtaining an accuracy over ~0.50 (let's say from ~0.51 on) in a validation or test set has some chances to get a positive score for that data.

In [ ]:
class_counts = full_y_train.value_counts()

px.bar(class_counts, title='Class distribution for training data', labels={'index': 'Class', 'value': 'Sample count', 'color': 'action'}, color=['1', '0'], text=class_counts.values)

## Operation number over time

The number of operation per days has a noteworthy variability: a standard deviation of about 1500 trades, for a mean of almost 4000; the peak is more than 13000 operations, while the minimum is set to only 11.

In [ ]:
import plotly.express as px

px.histogram(date_series, title='Number of operations per day', labels={'count': 'Number of operations', 'index': 'day'})

In [ ]:
print('Number of operations per day:')
date_series.value_counts().describe()

## Macro-level feature inspection

The features are examined at a macro-level before diving into fine-grained details.

Most features are floating-point numbers, with one integer one: feature 0. This feature can be actually considered categorical/binary, since it only takes two values: -1 and 1.

In [ ]:
full_x_train.dtypes.value_counts()

In [ ]:
full_x_train.select_dtypes(np.int32).columns

In [ ]:
full_x_train['feature_0'].value_counts()

All features except feature 0 have a large number of values, so they are not likely to be derived from nominal categories, but maybe one or few could be ordinal, such as the ones with lower cardinality, e.g. feature 43.

In [ ]:
full_x_train.nunique().sort_values()

The numeric variables have a similar mean, in the range [-0.1, 0.8]. However the maximum and minimum varies from one column to another, so they are not standarized (at least not with the usual standard deviation 1).

In [ ]:
col_mean = full_x_train.apply(lambda col: col.mean(), axis='rows')
print('Feature mean values:\n', col_mean.describe())

In [ ]:
col_extremes = full_x_train.apply(lambda col: col.agg([min, max]), axis='rows')
col_extremes

In [ ]:
total_min = col_extremes.min().min()
print('Global minimum:', total_min)

In [ ]:
total_max = col_extremes.max().max()
print('Global maximum:', total_max)

## Feature evolution over time

### Concepts and possible strategies

In time series, the independent and identically distributed (i.i.d.) data hypothesis is not usually met, with samples changing the statistics (mean, etc.) of their features over time. Hence, it is important to see if the raw features change over time in this dataset (and, if so, how much). If they do, it might be appropriate to use feature engineering or other mechanisms to derive features that account for their moment in time and extract the essential treats of samples so that prediction can be still meaningful even if new trend changes occur. For extracting the essence of features, auto-encoders could be used; the denoising auto-encoders are especially interesting due to the high noise that this trading dataset has, as well as variational auto-encoders, focused on extracting latent information from raw features.

Choosing the right algorithm is also important, of course. For instance, if one looks at the success of Recurrent Neural Networks (RNNs) in time series prediction problems, one sees that they leverage the relevance of recent samples (short memory) with key definining characteristics of older ones (long memory) to infer new responses. However, RNNs would be handy here if the test came just after the training in time, and for every new sample the previous ground-truth response was given, to update the model appropriately and avoid a massive deviation sonner. Hence, other algorithms, such as gradient boosting, which is state-of-the-art for many other tabular problems, could be preferred. Feed-forward neural networks with trainable or pre-trained auto-encoders may be a reasonable approach as well.


### Initial practical approach and findings

Every feature could be observed separately in order to find changes in trends over times. However, since there are many features, as an approach, one can try to examine all features together, being aware that clear oversimplifications will be used, and that other checks should follow to confirm any hypothesis. The approach used here is to scale all features to have similar numeric importance (mean 0, standard deviation 1). Then, the mean value over features (grouped and averaged by date) is compared from one date to another.

One can notice that there is indeed a divergence of mean value over time, starting in 0.04 to 0.06 in the first dozens of dates, reaching 0 near the day 120 and going down in an irregular manner, to finish with a value close to -0.04 by the final day (499), after a long period near to -0.02.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_x = pd.DataFrame(scaler.fit_transform(full_x_train), columns=full_x_train.columns)

In [ ]:
mean_scaled_value_per_day = scaled_x.mean(axis=1).groupby(date_series).mean()
mean_scaled_value_per_day.plot(title='Mean scaled feature value per day', ylabel='mean feature value')

In [ ]:
smoothed_mean_scaled_value_per_day = pd.Series(data=[np.mean(mean_scaled_value_per_day[max(0,i-20):i+1] if i >= 20 else np.nan) for i in range(len(mean_scaled_value_per_day))])
smoothed_mean_scaled_value_per_day.plot(title='Smoothed mean scaled feature value per day', ylabel='mean feature value', xlabel='date')

In [ ]:
mean_scaled_value_per_day.describe()

### Confirmation in specific features

As said before, the macro-level divergence was an oversimplification, that requried to confirm the hypothesis of significant change over time observing features individually. By doing so it can be confirmed that many features' mean values change over time in dramatic ways, which suggests that market/operation trends are very dynamic and have substantially different trends over a variety of time periods. While some features show a large change from the first 80-100 days compared to later dates, it can be observed that others have relevant changes in other periods of time, e.g. 68 has a large volatility over all the training period, while feature 113 has major peaks (followed/preceded by significant drops) between days 250 and 350 (other than in the 50-150 period).

In [ ]:
for i in range(13):
    full_x_train[['feature_' + str(10 * i + j) for j in range(10)]].groupby(date_series).mean().rolling(window=20).mean().plot(
        title='Rolling average of feature values over time (features {} to {})'.format(i * 10, i * 10 + 9), ylabel='Feature mean')

## Feature-target correlation

If any feature had a reasonably high correlation with the target, it would make the classification task easier. However, here the features are very uncorrelated with the target variable, with a maximum of 0.03 in positive correlation and 0.02 in negative one.

In [ ]:
correlations = pd.Series(data=[full_x_train[feature].corr(full_y_train) for feature in x_cols], 
                         index=[feature[len('feature_'):] for feature in x_cols])
print(correlations)

In [ ]:
print("Most target-correlated features:\n", correlations.sort_values(ascending=False).head())

In [ ]:
print("Least target-correlated features:\n", correlations.sort_values(ascending=True).head())

In [ ]:
fig = px.bar(correlations, title='Feature-target correlation for training data', labels={'index': 'Feature', 'value': 'Pearson correlation', 'variable': 'correlation'})
fig.update_layout(showlegend=False)
fig.show()

## Feature 43 inspection

You may recall from before that feature 43 has relatively low cardinality, so it might be a good idea to inspect if it shows any sign of representing categories instead of raw numbers.

Feature 43 is very noisy at a macro-level.

In [ ]:
full_x_train['feature_43'].plot()

Feature 43 is also very noisy within single a day.

In [ ]:
full_x_train['feature_43'][date_series[date_series == 0].index].plot()

If we see instead the mean by day, the values are quite different depending on the date; the early dates have much higher values. At least to this point of analysis, feature 43 does not seem to represent a trivial ordering or something obviously ordinal.

In [ ]:
full_x_train['feature_43'].groupby(date_series).mean().plot()

## Missing values and imputation options


On average, there are about 2% of missing values on average per feature, with one column exceeding 16%. Over half of features (88) have at least one missing value.

Different approaches can be taken to tackle the problem of missing data. Dropping affected columns or rows altogether would be a poor solution as a lot of non-missing information will be lost too. Except in some tree-based methods where missing values can be handled specially (e.g. auto-imputated by the algorithm), imputation should be the preferred approach. Per-column mean imputation could be considered, but since there is a lot of variability over time in some features, approaches such as same-date or recent-date based imputation (using mean, iterative prediction or something else) might give better results. It would be a good idea to try multiple options and see how well/bad they peform in cross validation.


In [ ]:
col_na_percent = full_x_train.apply(lambda col: col.isna().sum() / full_x_train.shape[0] * 100., axis='rows')
print('Percentage of missing values per feature:\n', col_na_percent)

In [ ]:
any_na_col_num = (col_na_percent > 0).sum()
print('Number of features with any missing values:', any_na_col_num)

In [ ]:
col_na_percent.describe()

## Further work

The exploratory analysis of this notebook is far from exhaustive, and many other aspects can be analyzed:

* Noise and the importance and contributions of features to explain variance (e.g. with PCA).
* Fine-grained per-feature analysis, to try to understand their meaning or relevance.
* Detailed micro-level (e.g. intra-day) analysis of missing values.
* Visualization of features in lower-dimensional spaces (e.g. with TSNE or UMAP).

Many of those aspects have been featured in other [notebooks of this competition](https://www.kaggle.com/c/jane-street-market-prediction/code).